In [1]:
# 一些常规特征
import pandas as pd
from tqdm.autonotebook import *
from bs4 import BeautifulSoup
import re
tqdm.pandas()

train = pd.read_csv('../data/train.csv')
test = pd.read_csv('../data/test.csv')
data = pd.concat([train, test], axis=0, sort=False).reset_index(drop=True)
print(data.columns)
# 将所有列进行拼接
data['review'] = data.progress_apply(lambda row:str(row['title']) + ' ' + str(row['department']) + ' ' + str(row['requirements']) + ' ' + str(row['benefits']), axis=1)

/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/ipykernel_launcher.py:3: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  This is separate from the ipykernel package so we can avoid doing imports until


Index(['title', 'location', 'department', 'salary_range', 'company_profile',
       'description', 'requirements', 'benefits', 'telecommuting',
       'has_company_logo', 'has_questions', 'employment_type',
       'required_experience', 'required_education', 'industry', 'function',
       'fraudulent'],
      dtype='object')


In [2]:
from gensim.models import Word2Vec
from collections import defaultdict
text_list = list(data['review'])

documents = text_list
texts = [[word for word in str(document).split(' ') ] for document in documents]
frequency = defaultdict(int)
for text in texts:
    for token in text:
        frequency[token] += 1
texts = [[token for token in text if frequency[token] >= 1] for text in texts]


w2v = Word2Vec(texts, size=32, seed=1017)
w2v.wv.save_word2vec_format('model/w2v_128_extend.txt')
print("w2v model done")

unable to import 'smart_open.gcs', disabling that module
/Users/yphacker/opt/anaconda3/envs/py36/lib/python3.6/site-packages/gensim/models/base_any2vec.py:743: UserWarning: C extension not loaded, training will be slow. Install a C compiler and reinstall gensim for fast training.
  "C extension not loaded, training will be slow. "


w2v model done


In [3]:

import gensim
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.text import CountVectorizer
import numpy as np

def get_w2v_avg(word2vec_Path,documents=documents):
    texts = documents
    w2v_dim = 32
    model = gensim.models.KeyedVectors.load_word2vec_format(word2vec_Path, binary=False)
    vacab = model.vocab.keys()
    w2v_feature = np.zeros((len(texts), w2v_dim))
    w2v_feature_avg = np.zeros((len(texts), w2v_dim))

    for i, line in enumerate(texts):
        num = 0
        if line == 'null':
            w2v_feature_avg[i, :] = np.zeros(w2v_dim)
        else:
            for word in line.split():
                num += 1
                if word=="":
                    vec =  np.zeros(w2v_dim)
                else:
                    vec = model[word] if word in vacab else np.zeros(w2v_dim)
                w2v_feature[i, :] += vec
            w2v_feature_avg[i, :] = w2v_feature[i, :] / num
    w2v_avg = pd.DataFrame(w2v_feature_avg)
    w2v_avg.columns = ['w2v_avg_' + str(i) for i in w2v_avg.columns]
    return w2v_avg
w2v_avg_feat=get_w2v_avg('model/w2v_128_extend.txt')
print("done")


done


In [4]:
w2v_avg_feat.to_csv('feature/w2v_extend_feature.csv', index=False)